<a href="https://colab.research.google.com/github/gc2321/ztm_pytorch/blob/main/miRNA_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


In [9]:
print(torch.__version__)

2.5.0+cu121


In [10]:
torch.cuda.is_available()

True

In [11]:
# set uo device agnoistic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# count number of devices
torch.cuda.device_count()

1


## Load Data

In [13]:
# mount the colab
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import fileinput
filePath = "/content/gdrive/MyDrive/neural_data/miRNA_lung.csv"

In [21]:
data = pd.read_csv(filePath)

In [23]:
data.head()

,depmap_id,cell_line_display_name,lineage_1,lineage_2,lineage_3,lineage_5,lineage_6,lineage_4,MIMAT0000062,MIMAT0000063,...,MIMAT0002192,MIMAT0002191,MIMAT0002190,MIMAT0002189,MIMAT0002188,MIMAT0002187,MIMAT0002186,MIMAT0002185,MIMAT0010151,MIMAT0010150
0,ACH-000382,CORL24,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.735474,12.091720,...,6.113534,6.609991,6.308885,6.262471,6.397461,5.966938,5.907852,6.572435,6.230741,6.910133
1,ACH-000187,CORL311,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.088938,10.298567,...,5.892877,6.482203,6.258330,6.425761,6.454340,6.050284,5.829088,6.321567,5.973841,6.723558
2,ACH-000695,CORL47,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.802760,12.660535,...,5.856488,6.421728,5.928844,6.166514,6.454669,6.084489,5.831370,6.084489,5.831370,6.735522
3,ACH-000743,CORL95,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.687715,12.203541,...,5.778997,6.364397,6.293885,6.181302,6.557043,6.219943,5.902315,6.464015,6.059182,6.779654
4,ACH-000594,DMS153,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,15.371449,13.265271,...,5.906890,6.691115,6.196332,6.455821,7.594922,6.106223,5.824513,6.380764,6.058966,7.831497


In [24]:
data['lineage'] = np.where(data['lineage_2'].str.contains('Non-small', case=False), 'Non-Small Cell', 'Small Cell')

In [25]:
data['lineage'].value_counts()

,count
lineage,
Non-Small Cell,119
Small Cell,49


In [26]:
columns_to_remove = ['depmap_id', 'cell_line_display_name', 'lineage_1', 'lineage_2', 'lineage_3', 'lineage_5', 'lineage_6', 'lineage_4']
data = data.drop(columns=columns_to_remove)

In [27]:
data.head(2)

,MIMAT0000062,MIMAT0000063,MIMAT0000064,MIMAT0000065,MIMAT0000066,MIMAT0000067,MIMAT0000414,MIMAT0000415,MIMAT0000416,MIMAT0000098,...,MIMAT0002191,MIMAT0002190,MIMAT0002189,MIMAT0002188,MIMAT0002187,MIMAT0002186,MIMAT0002185,MIMAT0010151,MIMAT0010150,lineage
0,14.735474,12.091720,9.961391,12.626818,9.808289,12.016231,12.647656,12.811913,6.214513,6.262471,...,6.609991,6.308885,6.262471,6.397461,5.966938,5.907852,6.572435,6.230741,6.910133,Small Cell
1,14.088938,10.298567,9.188911,9.433690,7.415911,9.548706,10.592279,10.094434,6.192391,6.209064,...,6.482203,6.258330,6.425761,6.454340,6.050284,5.829088,6.321567,5.973841,6.723558,Small Cell


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Columns: 735 entries, MIMAT0000062 to lineage
dtypes: float64(734), object(1)
memory usage: 964.8+ KB


In [29]:
# Check if any cell contains NaN
any_nan = data.isna().any().any()
any_nan

False

In [30]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)